# <font color='blue'> **Data Sprint HDB Resale Prices** </font>

# <font color='red'> **This file for inserting column 'Predicted_Price' into test.csv**

### Only cleaning, using loaded Random forest model & exporting here.

### Ameer, Benjamin, Leonard, Wei Yu.

---

---

# Importing libraries and CSV file

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from scipy.stats import iqr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import geopandas as gpd
import re
import streamlit as st
import joblib


%matplotlib inline

In [2]:
hdb = pd.read_csv('./datasets/test.csv')
hdb.insert(10, 'Predicted_Price', 0)

hdb.head()

/var/folders/bv/9kp7wlwx1n94x819spyh3pzc0000gn/T/ipykernel_1258/1479000812.py:1: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  hdb = pd.read_csv('./datasets/test.csv')


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.0,Simplified,1987,...,92,0,1.433681,103.832924,156.322353,Ahmad Ibrahim Secondary School,218,0,1.436235,103.829987
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,2008,...,45,0,1.339244,103.698896,739.371688,Jurong West Secondary School,199,0,1.335256,103.702098
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,36,0,1.371893,103.851811,305.071191,Anderson Secondary School,245,0,1.374242,103.851430
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.0,New Generation,1979,...,54,0,1.434423,103.773698,433.454591,Woodlands Secondary School,188,0,1.439183,103.774499
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,1985,...,40,0,1.349195,103.741000,217.295361,Bukit Batok Secondary School,223,0,1.348351,103.740873


# Dropping irrelevant columns after inspecting Data dict

In [3]:
hdb = hdb.drop(columns=['block', 'address', 'lower', 'upper', 'mid', 'full_flat_type', 'floor_area_sqft', 'lease_commence_date', 'year_completed', 
                  'postal', 'Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km', 'Hawker_Within_500m', 'Hawker_Within_1km', 
                  'Hawker_Within_2km', 'mrt_name', 'mrt_latitude', 'mrt_longitude', 'bus_stop_name', 'bus_stop_latitude', 'bus_stop_longitude',
                  'pri_sch_name', 'vacancy', 'pri_sch_latitude', 'pri_sch_longitude', 'sec_sch_name', 'affiliation', 'sec_sch_latitude', 
                  'sec_sch_longitude', 'planning_area', 'residential', 'market_hawker'])

# Checking for NAs

In [4]:
hdb.isna().sum().sort_values(ascending=False).head()

Mall_Nearest_Distance    84
id                        0
5room_sold                0
multigen_sold             0
studio_apartment_sold     0
dtype: int64

# Imputing 829 NAs in Mall_Nearest_Distance

In [5]:
mean_nearest_mall = hdb.groupby('town')['Mall_Nearest_Distance'].mean()

mean_nearest_mall = mean_nearest_mall.reset_index()

In [6]:
hdb = pd.merge(hdb, mean_nearest_mall, how='left', on='town')

In [7]:
hdb.head()

,id,Tranc_YearMonth,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,Predicted_Price,Tranc_Year,...,hawker_market_stalls,mrt_nearest_distance,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,Mall_Nearest_Distance_y
0,114982,2012-11,YISHUN,4 ROOM,YISHUN AVE 7,07 TO 09,84.0,Simplified,0,2012,...,123,686.660434,0,0,75.683952,426.467910,0,156.322353,218,616.483084
1,95653,2019-08,JURONG WEST,5 ROOM,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,0,2019,...,94,169.478175,0,0,88.993058,439.756851,0,739.371688,199,619.560415
2,40303,2013-10,ANG MO KIO,3 ROOM,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,0,2013,...,100,694.220448,1,0,86.303575,355.882207,0,305.071191,245,771.042907
3,109506,2017-10,WOODLANDS,4 ROOM,MARSILING DR,01 TO 03,97.0,New Generation,0,2017,...,112,1117.203587,0,0,108.459039,929.744711,0,433.454591,188,556.388392
4,100149,2016-08,BUKIT BATOK,4 ROOM,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,0,2016,...,87,987.976010,0,0,113.645431,309.926934,0,217.295361,223,908.945339


In [8]:
hdb['Mall_Nearest_Distance_x'] = hdb['Mall_Nearest_Distance_x'].fillna(hdb['Mall_Nearest_Distance_y'])

In [9]:
hdb = hdb.drop(columns='Mall_Nearest_Distance_y')

In [10]:
hdb.isnull().sum().sort_values(ascending=False).head()

id                       0
5room_sold               0
multigen_sold            0
studio_apartment_sold    0
1room_rental             0
dtype: int64

# Checking datatypes

In [11]:
hdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         16737 non-null  int64  
 1   Tranc_YearMonth            16737 non-null  object 
 2   town                       16737 non-null  object 
 3   flat_type                  16737 non-null  object 
 4   street_name                16737 non-null  object 
 5   storey_range               16737 non-null  object 
 6   floor_area_sqm             16737 non-null  float64
 7   flat_model                 16737 non-null  object 
 8   Predicted_Price            16737 non-null  int64  
 9   Tranc_Year                 16737 non-null  int64  
 10  Tranc_Month                16737 non-null  int64  
 11  mid_storey                 16737 non-null  int64  
 12  price_per_sqft             16737 non-null  float64
 13  hdb_age                    16737 non-null  int

In [12]:
hdb['Tranc_YearMonth'] = pd.to_datetime(hdb['Tranc_YearMonth'], infer_datetime_format=True)

/var/folders/bv/9kp7wlwx1n94x819spyh3pzc0000gn/T/ipykernel_1258/4027540773.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  hdb['Tranc_YearMonth'] = pd.to_datetime(hdb['Tranc_YearMonth'], infer_datetime_format=True)


In [13]:
hdb = hdb.astype({'floor_area_sqm':'float'})

In [14]:
# commercial
# multistorey_carpark
# precinct_pavilion

# These 3 columns have 'Y' & 'N' values. We change them to 1's & 0's. 

hdb['commercial'] = hdb['commercial'].map({'Y': 1, 'N' : 0})

hdb['multistorey_carpark'] = hdb['multistorey_carpark'].map({'Y': 1, 'N' : 0})

hdb['precinct_pavilion'] = hdb['precinct_pavilion'].map({'Y': 1, 'N' : 0})

In [15]:
hdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         16737 non-null  int64         
 1   Tranc_YearMonth            16737 non-null  datetime64[ns]
 2   town                       16737 non-null  object        
 3   flat_type                  16737 non-null  object        
 4   street_name                16737 non-null  object        
 5   storey_range               16737 non-null  object        
 6   floor_area_sqm             16737 non-null  float64       
 7   flat_model                 16737 non-null  object        
 8   Predicted_Price            16737 non-null  int64         
 9   Tranc_Year                 16737 non-null  int64         
 10  Tranc_Month                16737 non-null  int64         
 11  mid_storey                 16737 non-null  int64         
 12  pric

# Handling outliers:

In [16]:
mature_estates = pd.read_csv('./datasets/mature estates.csv')

mature_estates.head()

,town,Mature Estates,zone
0,ANG MO KIO,1,north
1,BEDOK,1,east
2,BISHAN,1,north
3,BUKIT MERAH,1,south
4,BUKIT TIMAH,1,west


In [17]:
hdb = pd.merge(hdb, mature_estates, how='left', on='town')

In [18]:
hdb.head()

,id,Tranc_YearMonth,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,Predicted_Price,Tranc_Year,...,mrt_nearest_distance,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,Mature Estates,zone
0,114982,2012-11-01,YISHUN,4 ROOM,YISHUN AVE 7,07 TO 09,84.0,Simplified,0,2012,...,686.660434,0,0,75.683952,426.467910,0,156.322353,218,0.0,north
1,95653,2019-08-01,JURONG WEST,5 ROOM,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,0,2019,...,169.478175,0,0,88.993058,439.756851,0,739.371688,199,0.0,west
2,40303,2013-10-01,ANG MO KIO,3 ROOM,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,0,2013,...,694.220448,1,0,86.303575,355.882207,0,305.071191,245,1.0,north
3,109506,2017-10-01,WOODLANDS,4 ROOM,MARSILING DR,01 TO 03,97.0,New Generation,0,2017,...,1117.203587,0,0,108.459039,929.744711,0,433.454591,188,0.0,north
4,100149,2016-08-01,BUKIT BATOK,4 ROOM,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,0,2016,...,987.976010,0,0,113.645431,309.926934,0,217.295361,223,0.0,west


---

# One-hot encoding

In [19]:
# drop_first argument to save coding lines 
hdb = pd.get_dummies(hdb, columns=['zone'], drop_first=True)

In [20]:
hdb.head()

,id,Tranc_YearMonth,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,Predicted_Price,Tranc_Year,...,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_affiliation,sec_sch_nearest_dist,cutoff_point,Mature Estates,zone_east,zone_north,zone_south,zone_west
0,114982,2012-11-01,YISHUN,4 ROOM,YISHUN AVE 7,07 TO 09,84.0,Simplified,0,2012,...,75.683952,426.467910,0,156.322353,218,0.0,False,True,False,False
1,95653,2019-08-01,JURONG WEST,5 ROOM,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,0,2019,...,88.993058,439.756851,0,739.371688,199,0.0,False,False,False,True
2,40303,2013-10-01,ANG MO KIO,3 ROOM,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,0,2013,...,86.303575,355.882207,0,305.071191,245,1.0,False,True,False,False
3,109506,2017-10-01,WOODLANDS,4 ROOM,MARSILING DR,01 TO 03,97.0,New Generation,0,2017,...,108.459039,929.744711,0,433.454591,188,0.0,False,True,False,False
4,100149,2016-08-01,BUKIT BATOK,4 ROOM,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,0,2016,...,113.645431,309.926934,0,217.295361,223,0.0,False,False,False,True


---

# Load trained model, make predictions

In [ ]:
# Select input features
input_features = [
    'floor_area_sqm', 'exec_sold', '5room_sold', 'max_floor_lvl', 'hdb_age',
    'total_dwelling_units', '3room_sold',
    'zone_east', 'zone_north', 'zone_south', 'zone_west'
]



# Drop rows with missing values in input features
hdb = hdb.dropna(subset=input_features)

# Prepare the input DataFrame (preserves column names)
X = hdb[input_features]

# Load model
loaded_model = joblib.load(r'./random_forest_model.pkl')

# Predict
predictions = loaded_model.predict(X)  # ✅ no warning here


# Add predictions to the DataFrame as a new column
hdb['Predicted_Price'] = predictions



# Export csv with predictions column 'Predicted_Price' for test.csv

In [22]:
# Save the DataFrame (with predictions) to a CSV file
hdb.to_csv(r'hdb_with_predictions.csv', index=False)

---